<h1>Notebook di prova per Altair</h1>

<h2>Caricamento delle librerie e il dataset</h2>

In [1]:
#!pip install vega

<p>Import delle librerie necessarie</p>

In [2]:
import pandas as pd
import altair as alt
import altair_viewer as av
from altair import datum
import re
import string
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tokenize import word_tokenize

In [3]:
alt.__version__

'5.0.0'

<p>carico il dataset</p>

In [4]:
df = pd.read_csv('datasetForAltair-eng (1).csv', index_col=0, low_memory=False)

In [5]:
df2 = pd.read_csv('dfITAltair.csv', index_col=0, low_memory=False)

In [6]:
df.head(3)

,user,date,likeCount,tweetText,hashtags,lang,replies,retweetCount,replyCount,Year,Month,Day,WeekDay,hour,minutes,dayofYear,date_only,tweetText_proc,Sentiment
0,jdemay,2023-02-05 23:59:16+00:00,0,Planning to use #ChatGPT to help you with high...,['ChatGPT'],en,0.0,0.0,NaN,2023,2,5,Sunday,23,59,36,2023-02-05,planning use help high school math ? maybe thi...,pos
1,CarlosSilvaB81,2023-02-05 23:58:51+00:00,3,"@Carmtans @stkirsch Chat gpt, is nothing but a...",NaN,en,0.0,0.0,NaN,2023,2,5,Sunday,23,58,36,2023-02-05,"chat gpt , nothing super search engine , progr...",neg
2,saadnajeebsaad,2023-02-05 23:58:15+00:00,0,#WeatherUpdate #Karachi #Khi #TempUpdate #Sunr...,"['WeatherUpdate', 'Karachi', 'Khi', 'TempUpdat...",en,0.0,0.0,NaN,2023,2,5,Sunday,23,58,36,2023-02-05,sunrise : 07:12 sunset : 06:19 pm current temp...,neu


In [7]:
df2.head()

,user,date,likeCount,tweetText,hashtags,lang,replies,retweetCount,replyCount,Year,Month,Day,WeekDay,hour,minutes,dayofYear,date_only,tweetText_proc,Sentiment
0,40annibuttati,2023-02-05 23:17:31+00:00,0,Interessante riflessione: \ni contribuenti ita...,"['Alitalia', 'Microsoft', 'ChatGPT']",it,0.0,0.0,NaN,2023,2,5,NaN,23,17,36,2023-02-05,interessante riflessione : contribuenti italia...,pos
1,PakiAsWell,2023-02-05 23:15:41+00:00,0,Quando ti spari le pose con l'AI ... senza dir...,"['ChatGPT', 'AppleDeveloperAcademy', 'swiftUI'...",it,0.0,0.0,NaN,2023,2,5,NaN,23,15,36,2023-02-05,quando spari pose l'ai ... senza dirgli resto ...,neu
2,GbshiatsuM,2023-02-05 23:12:22+00:00,0,@yarotrof Chat gpt non ragiona bensì riprende ...,NaN,it,0.0,0.0,NaN,2023,2,5,NaN,23,12,36,2023-02-05,chat gpt ragiona bensì riprende informazioni o...,neu
3,finmoderna,2023-02-05 22:42:09+00:00,1,Azioni intelligenza artificiale: 15 titoli da ...,"['ai', 'ia', 'intelligenzaartificiale', 'artif...",it,0.0,0.0,NaN,2023,2,5,NaN,22,42,36,2023-02-05,azioni intelligenza artificiale : 15 titoli te...,neu
4,Tarallucci_Vino,2023-02-05 21:49:54+00:00,2,🙄LA FINE DEL CAPITALISMO\n\nSe speravate di as...,NaN,it,3.0,0.0,NaN,2023,2,5,NaN,21,49,36,2023-02-05,🙄la fine capitalismo speravate assistere tramo...,pos


<p>stampo info relative al dataset corrente</p>

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287309 entries, 0 to 287308
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user            280601 non-null  object 
 1   date            287309 non-null  object 
 2   likeCount       287309 non-null  int64  
 3   tweetText       287309 non-null  object 
 4   hashtags        177869 non-null  object 
 5   lang            287309 non-null  object 
 6   replies         281713 non-null  float64
 7   retweetCount    287202 non-null  float64
 8   replyCount      5596 non-null    float64
 9   Year            287309 non-null  int64  
 10  Month           287309 non-null  int64  
 11  Day             287309 non-null  int64  
 12  WeekDay         287309 non-null  object 
 13  hour            287309 non-null  int64  
 14  minutes         287309 non-null  int64  
 15  dayofYear       287309 non-null  int64  
 16  date_only       287309 non-null  object 
 17  tweetText_

<h2>Processing dei dati testuali</h2>

In [9]:
def pre_process(text):
    if isinstance(text, str):
        # Remove links
        text = re.sub(r'http\S+', '', text)

        # Remove mentions and hashtags
        text = re.sub(r'@\w+|#\w+', '', text)

        # Remove punctuation marks
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word.lower() not in stop_words]

        # Remove empty and short words
        tokens = [word for word in tokens if len(word) > 2]

        # Join the tokens back into a single string
        cleaned_text = ' '.join(tokens)

        return cleaned_text
    else:
        return ''

In [10]:
#applico la pulizia del testo
df['tweetText_proc'] = df['tweetText_proc'].apply(pre_process)

In [11]:
#applico la pulizia del testo
df2['tweetText_proc'] = df2['tweetText_proc'].apply(pre_process)

In [12]:
print("Sentiment sul df inglese")
print(df['Sentiment'].value_counts())
print("Sentiment sul df italiano")
print(df2['Sentiment'].value_counts())

Sentiment sul df inglese
pos    142430
neu     95570
neg     49309
Name: Sentiment, dtype: int64
Sentiment sul df italiano
neu    16899
pos     2991
neg     1160
Name: Sentiment, dtype: int64


In [13]:
#creo un mapping del sentiment per avere i nomi completi delle etichette

In [14]:
sentiment_mapping = {'pos': 'Positivo', 'neu': 'Neutrale', 'neg': 'Negativo'}

df['Sentimento'] = df['Sentiment'].map(sentiment_mapping)
df2['Sentimento'] = df2['Sentiment'].map(sentiment_mapping)

In [15]:
df['Sentimento'].value_counts()

Positivo    142430
Neutrale     95570
Negativo     49309
Name: Sentimento, dtype: int64

In [16]:
df2['Sentimento'].value_counts()

Neutrale    16899
Positivo     2991
Negativo     1160
Name: Sentimento, dtype: int64

<h2>Analisi del sentimento attraverso i mesi nei commenti di twitter</h2>

In [17]:

# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Group by month and sentiment
df_grouped = df.groupby([df['date'].dt.to_period('M'), 'Sentimento']).size().unstack()

# Convert index to string and extract month names
df_grouped.index = df_grouped.index.strftime('%B %Y')

# Reshape the dataframe for Altair
df_grouped = df_grouped.reset_index().melt('date', var_name='Sentimento', value_name='Value')

# Define color domain and range
domain = ['Positivo', 'Neutrale', 'Negativo']
range_ = ['#ADFC92', '#788BFF', '#F44E3F']

# Create the Altair chart
chart = alt.Chart(df_grouped, title=alt.Title(
       "Evoluzione del sentimento verso ChatGPT nell'arco dei mesi",
       subtitle="Visualizzazione dei valori relativi al dataset di Tweets"
   )).mark_line(opacity=0.1).encode(
    x=alt.X('date:T', axis=alt.Axis(title='Periodo', format='%B %Y', tickCount=12)),
    y=alt.Y('Value:Q', axis=alt.Axis(title='Numero di tweets')),
    opacity=alt.value(0.9),
    color=alt.Color('Sentimento',
                    title="Sentimento registrato",
                    scale=alt.Scale(domain=domain, range=range_),
                    legend=alt.Legend(orient="right"))
).configure_legend(
    orient='top'
).configure_axis(
    grid=False
).properties(
    width=800,
    height=600
).interactive()

chart.save('chartSentimentING.html')
chart

C:\Users\claud\AppData\Local\Temp\ipykernel_27376\2417200938.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_grouped = df.groupby([df['date'].dt.to_period('M'), 'Sentimento']).size().unstack()


alt.Chart(...)

In [18]:

# Convert 'date' column to datetime
df2['date'] = pd.to_datetime(df2['date'])

# Group by month and sentiment
df_grouped = df2.groupby([df2['date'].dt.to_period('M'), 'Sentimento']).size().unstack()

# Convert index to string and extract month names
df_grouped.index = df_grouped.index.strftime('%B %Y')

# Reshape the dataframe for Altair
df_grouped = df_grouped.reset_index().melt('date', var_name='Sentimento', value_name='Value')

# Define color domain and range
domain = ['Positivo', 'Neutrale', 'Negativo']
range_ = ['#ADFC92', '#788BFF', '#F44E3F']

# Create the Altair chart
chart2 = alt.Chart(df_grouped, title=alt.Title(
       "Evoluzione del sentimento verso ChatGPT nell'arco dei mesi",
       subtitle="Visualizzazione dei valori relativi al dataset di tweets italiani"
   )).mark_line(opacity=0.1).encode(
    x=alt.X('date:T', axis=alt.Axis(title='Periodo', format='%B %Y', tickCount=12)),
    y=alt.Y('Value:Q', axis=alt.Axis(title='Numero di tweets')),
    opacity=alt.value(0.9),
    color=alt.Color('Sentimento',
                    title="Sentimento registrato",
                    scale=alt.Scale(domain=domain, range=range_),
                    legend=alt.Legend(orient="right"))
).configure_legend(
    orient='top'
).configure_axis(
    grid=False
).properties(
    width=800,
    height=600
).interactive()

chart2.save('chartITsentiment.html')
chart2

C:\Users\claud\AppData\Local\Temp\ipykernel_27376\797412524.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_grouped = df2.groupby([df2['date'].dt.to_period('M'), 'Sentimento']).size().unstack()


alt.Chart(...)

In [19]:
def extract_year(df, value):
    return df[df['Year']==value]

y22 = extract_year(df, 2022)
y23 = extract_year(df, 2023)

In [20]:
y22IT = extract_year(df2, 2022)
y23IT = extract_year(df2, 2023)

In [21]:
print ("dimensioni dataset 2022 (righe x colonne): "+ str(y22.shape))
print ("dimensioni dataset 2023 (righe x colonne): "+ str(y23.shape))

dimensioni dataset 2022 (righe x colonne): (77338, 20)
dimensioni dataset 2023 (righe x colonne): (209971, 20)


In [22]:
print ("dimensioni dataset italiano 2022 (righe x colonne): "+ str(y22IT.shape))
print ("dimensioni dataset italiano 2023 (righe x colonne): "+ str(y23IT.shape))

dimensioni dataset italiano 2022 (righe x colonne): (1473, 20)
dimensioni dataset italiano 2023 (righe x colonne): (19577, 20)


In [23]:
#prendo il dataset, cerco le frequenze delle parole per sentimento (=risultato: tre dataframe, uno per sentimento e relativi a un anno.)
#creo un dataset con 2 colonne (uno per ogni sentimento), le colonne sono: word, frequency
#creo una visualizzazione scatter plot con colore abbinato al sentimento corrente
#mostro tutti i puntini usando un Alt che in descrizione riporta il valore label di word
#mostro una legenda con le top 20 più frequenti

In [24]:
#prendo il dataset, cerco le frequenze delle parole per sentimento.
def words_finder(df, sentiment):
    tokens = []
    df = df[(df['Sentimento'] == sentiment)]
    for row in df['tweetText_proc']:
        tokens.extend(word_tokenize(str(row).lower()))  # Tokenization and lower conversion of the strings

    word_freq = FreqDist(tokens)
    sorted_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    banned_w = ['chatgpt', 'ai', ' ai', 'ai ', 'gpt', 'openai', 'artificialintelligence', 'amp', 'https', '2022', '2023', 'also', 'still', 'could', 'può', 'ancora', 'dopo', 'sempre','però', 'poi', 'così'] #words to filter out
    sorted_freq = [(word, freq) for word, freq in sorted_freq if word not in banned_w]
    return sorted_freq #frequencies sorted for the choosen sentiment

def createDffreqs(sorted_freq, sentiment): #ritorna un dataframe con una colonna contenente la parola ed una con la sua frequenza assoluta
    freq_df = pd.DataFrame(sorted_freq, columns=['Parola', 'Frequenza'])
    freq_df.sort_values(by='Frequenza', ascending=False, inplace=True)
    freq_df['Sentimento'] = sentiment
    return freq_df

In [25]:
y22pos = createDffreqs(words_finder(y22, 'Positivo'), 'Positivo') #dataframe commenti positivi del 2022
top22pos = y22pos.head(30)

In [26]:
y23pos = createDffreqs(words_finder(y23, 'Positivo'), 'Positivo') #dataframe commenti positivi del 2023
top23pos = y23pos.head(30)

In [27]:
y22neg = createDffreqs(words_finder(y22, 'Negativo'), 'Negativo') #dataframe commenti negativi del 2022
top22neg = y22neg.head(30)

In [28]:
y23neg = createDffreqs(words_finder(y23, 'Negativo'), 'Negativo') #dataframe commenti negativi del 2023
top23neg = y23neg.head(30)

In [29]:
y22neu = createDffreqs(words_finder(y22, 'Neutrale'), 'Neutrale') #dataframe commenti neutri del 2022
top22neu = y22neu.head(30)

In [30]:
y23neu = createDffreqs(words_finder(y23, 'Neutrale'), 'Neutrale') #dataframe di commenti neutri del 2023
top23neu = y23neu.head(30)

In [31]:
y22posIT = createDffreqs(words_finder(y22IT, 'Positivo'), 'Positivo') #dataframe commenti positivi del 2022
top22ITpos = y22posIT.head(30)

In [32]:
y23posIT = createDffreqs(words_finder(y23IT, 'Positivo'), 'Positivo') #dataframe commenti positivi del 2023
top23ITpos = y23posIT.head(30)

In [33]:
y22negIT = createDffreqs(words_finder(y22IT, 'Negativo'), 'Negativo') #dataframe commenti negativi del 2022
top22ITneg = y22negIT.head(30)

In [34]:
y23negIT = createDffreqs(words_finder(y23IT, 'Negativo'), 'Negativo') #dataframe commenti negativi del 2023
top23ITneg = y23negIT.head(30)

In [35]:
y22neuIT = createDffreqs(words_finder(y22IT, 'Neutrale'), 'Neutrale') #dataframe commenti neutri del 2022
top22ITneu = y22neuIT.head(30)

In [36]:
y23neuIT = createDffreqs(words_finder(y23IT, 'Neutrale'), 'Neutrale') #dataframe di commenti neutri del 2023
top23ITneu = y23neuIT.head(30)

In [37]:
y23neuIT

,Parola,Frequenza,Sentimento
0,artificiale,2054,Neutrale
1,chat,1744,Neutrale
2,intelligenza,1333,Neutrale
3,cosa,943,Neutrale
4,google,860,Neutrale
...,...,...,...
13834,artificialeconcordo,1,Neutrale
13835,replicarlanon,1,Neutrale
13836,avverto,1,Neutrale
13837,leggibile,1,Neutrale


In [110]:
# Concatenate the three DataFrames into a single DataFrame
df22words = pd.concat([top22pos, top22neg, top22neu])#, keys=['Positive', 'Negative', 'Neutral'])
df22words = df22words.groupby(['Sentimento', 'Parola']).agg({'Frequenza': 'sum'}).reset_index().set_index('Sentimento')
df22ITwords = pd.concat([top22ITpos, top22ITneg, top22ITneu])#, keys=['Positivo', 'Negativo', 'Neutrale'])
df22ITwords = df22ITwords.groupby(['Sentimento', 'Parola']).agg({'Frequenza': 'sum'}).reset_index().set_index('Sentimento')
options = ['Positivo', 'Neutrale', 'Negativo']
labels = [option + ' ' for option in options]

# Define color domain and range
domain = ['Positivo', 'Neutrale', 'Negativo']
range_ = ['#ADFC92', '#788BFF', '#F44E3F']

input_dropdown = alt.binding_radio(
    # Add the empty selection which shows all when clicked
    options=options + [None],
    labels=labels + ['Tutto'],
    name='Selettore sentimento: '
)

selection = alt.selection_point(
    fields=['Sentimento'],
    bind=input_dropdown,
)

# Reset the index of the concatenated DataFrame
df22words = df22words.reset_index(level=0).rename(columns={'level_0': 'Sentimento'})
alt.data_transformers.disable_max_rows()
scatter_plot_2022 = alt.Chart(df22words).mark_point(filled=True).encode(
    alt.X('Parola',title="Parola"),
    alt.Y('Frequenza', title="Frequenza").axis(labels=False),
    alt.Size('Frequenza'),
    alt.OpacityValue(0.7),
    alt.Color('Sentimento', title="Sentimento registrato").scale(domain=options, range=range_),
    tooltip = [alt.Tooltip('Parola:N'),
               alt.Tooltip('Frequenza:Q'),
               alt.Tooltip('Frequenza:Q'),
               #alt.Tooltip('Sentiment:Q')
              ]
).add_params(
    selection
).transform_filter(
    selection
).configure_axisX(
    labelAngle=45
).properties(
    width=800,  # Set the width of the chart to your desired value
    height=600,  # Set the height of the chart to your desired value
    title='Sentimento sulle prime 30 parole di tweets inglesi nel 2022'
).interactive()

df22ITwords = df22ITwords.reset_index(level=0).rename(columns={'level_0': 'Sentimento'})

scatter_plot_2022_ita = alt.Chart(df22ITwords).mark_point(filled=True).encode(
    alt.X('Parola',title="Parola"),
    alt.Y('Frequenza', title="Frequenza").axis(labels=False),
    alt.Size('Frequenza'),
    alt.OpacityValue(0.7),
    alt.Color('Sentimento', title="Sentimento registrato").scale(domain=options, range=range_),
    tooltip = [alt.Tooltip('Parola:N'),
               alt.Tooltip('Frequenza:Q'),
               alt.Tooltip('Frequenza:Q'),
               #alt.Tooltip('Sentiment:Q')
              ]
).add_params(
    selection
).transform_filter(
    selection
).configure_axisX(
    labelAngle=45
).properties(
    width=800,  # Set the width of the chart to your desired value
    height=600,  # Set the height of the chart to your desired value
    title='Sentimento sulle prime 30 parole di tweets italiani nel 2022'
).interactive()

scatter_plot_2022.save('chart-wordsfrequency-2022.html')
scatter_plot_2022

alt.Chart(...)

In [111]:
scatter_plot_2022_ita.save('chart-wordsfrequency-2022-ita.html')
scatter_plot_2022_ita

alt.Chart(...)

In [114]:
# Concatenate the three DataFrames into a single DataFrame
df23words = pd.concat([top23pos, top23neg, top23neu])#, keys=['Positive', 'Negative', 'Neutral'])
df23words = df23words.groupby(['Sentimento', 'Parola']).agg({'Frequenza': 'sum'}).reset_index().set_index('Sentimento')
df23ITwords = pd.concat([top23ITpos, top23ITneg, top23ITneu])#, keys=['Positive', 'Negative', 'Neutral'])
df23ITwords = df23ITwords.groupby(['Sentimento', 'Parola']).agg({'Frequenza': 'sum'}).reset_index().set_index('Sentimento')
options = ['Positivo', 'Neutrale', 'Negativo']
labels = [option + ' ' for option in options]

# Define color domain and range
domain = ['Positivo', 'Neutrale', 'Negativo']
range_ = ['#ADFC92', '#788BFF', '#F44E3F']

input_dropdown = alt.binding_radio(
    # Add the empty selection which shows all when clicked
    options=options + [None],
    labels=labels + ['Tutto'],
    name='Selettore sentimento 2023: '
)

selection = alt.selection_point(
    fields=['Sentimento'],
    bind=input_dropdown,
)

# Reset the index of the concatenated DataFrame
df23words = df23words.reset_index(level=0).rename(columns={'level_0': 'Sentimento'})
alt.data_transformers.disable_max_rows()
scatter_plot_2023 = alt.Chart(df23words).mark_point(filled=True).encode(
    alt.X('Parola', title="Parola"),
    alt.Y('Frequenza', title="Frequenza").axis(labels=False),
    alt.Size('Frequenza', title="Frequenza"),
    alt.OpacityValue(0.7),
    alt.Color('Sentimento', title="Sentimento registrato").scale(domain=options, range=range_),
    tooltip = [alt.Tooltip('Parola:N', title="Parola"),
               alt.Tooltip('Frequenza:Q', title="Frequenza")
               #alt.Tooltip('Sentiment:Q')
              ]
).add_params(
    selection
).configure_axisX(
    labelAngle=45
).transform_filter(
    selection
).properties(
    width=800,  # Set the width of the chart to your desired value
    height=600,  # Set the height of the chart to your desired value
    title='Sentimento sulle prime 30 parole di tweets inglesi nel 2023'
).interactive()

# Reset the index of the concatenated DataFrame
df23ITwords = df23ITwords.reset_index(level=0).rename(columns={'level_0': 'Sentimento'})
alt.data_transformers.disable_max_rows()
scatter_plot_2023_ita = alt.Chart(df23ITwords).mark_point(filled=True).encode(
    alt.X('Parola', title="Parola"),
    alt.Y('Frequenza', title="Frequenza").axis(labels=False),
    alt.Size('Frequenza', title="Frequenza"),
    alt.OpacityValue(0.7),
    alt.Color('Sentimento', title="Sentimento registrato").scale(domain=options, range=range_),
    tooltip = [alt.Tooltip('Parola:N', title="Parola"),
               alt.Tooltip('Frequenza:Q', title="Frequenza")
               #alt.Tooltip('Sentiment:Q')
              ]
).add_params(
    selection
).configure_axisX(
    labelAngle=45
).transform_filter(
    selection
).properties(
    width=800,  # Set the width of the chart to your desired value
    height=600,  # Set the height of the chart to your desired value
    title='Sentimento sulle prime 30 parole di tweets italiani nel 2023'
).interactive()

scatter_plot_2023.save('chart-wordsfrequency-2023.html')
scatter_plot_2023

alt.Chart(...)

In [115]:
scatter_plot_2023_ita.save('chart-wordsfrequency-2023-ita.html')
scatter_plot_2023_ita

alt.Chart(...)

In [43]:
#finalchart_wordsfrequency_2022_2023 = alt.hconcat(scatter_plot_2022, scatter_plot_2023).resolve_scale(y='independent'
#).configure_axisX(
#    labelAngle=45,  # Specify the degree of inclination here
#    labelAlign='left',  # Optional: Align the labels to the right for better readability
#    labelPadding=15  # Optional: Add padding to the labels for better spacing
#)
#finalchart_wordsfrequency_2022_2023.save('wordsfreq_2022_2023.html')
#finalchart_wordsfrequency_2022_2023
#finalchart_wordsfrequency_2022_2023_ita = alt.hconcat(scatter_plot_2022_ita, scatter_plot_2023_ita).resolve_scale(y='independent'
#).configure_axisX(
#    labelAngle=45,  # Specify the degree of inclination here
#    labelAlign='left',  # Optional: Align the labels to the right for better readability
#3    labelPadding=15  # Optional: Add padding to the labels for better spacing
#)
#finalchart_wordsfrequency_2022_2023_ita.save('wordsfreq_2022_2023_ita.html')
#finalchart_wordsfrequency_2022_2023_ita

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize the sentiment analysis model
sia = SentimentIntensityAnalyzer()

In [51]:
def get_top_n_ngrams(corpus, n=None, ngram=2, exclude_keywords=None):
    if exclude_keywords is None:
        exclude_keywords = []
        
    vec = CountVectorizer(ngram_range=(ngram, ngram), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

    # escludo le parole forbidden (poco rilevanti)
    words_freq = [item for item in words_freq if not any(keyword in item[0] for keyword in exclude_keywords)]

    top_ngrams = words_freq[:n]
    return top_ngrams

# raggruppo i trigrammi per mese 
top_trigrams_by_month = df.groupby('Month')['tweetText_proc'].apply(lambda x: get_top_n_ngrams(x, n=10, ngram=3, exclude_keywords=['chat', 'gpt', 'chatgpt','httpstcorlyimpqw', 'ai', 'eth']))

# creo un df per metterci i trigrammi e il mese
df_top_trigrams = pd.DataFrame(top_trigrams_by_month.reset_index())

# espando la lista dei trigrammi su rows distinte
df_top_trigrams = df_top_trigrams.explode('tweetText_proc')

# Split the 'tweet_text' in trigram e frequency
df_top_trigrams[['Trigram', 'Frequency']] = pd.DataFrame(df_top_trigrams['tweetText_proc'].tolist(), index=df_top_trigrams.index)


df_top_trigrams = df_top_trigrams.drop(columns='tweetText_proc')


df_top_trigrams['Sentimento'] = df_top_trigrams['Trigram'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [52]:
# raggruppo i trigrammi per mese 
top_trigrams_by_month_it = df2.groupby('Month')['tweetText_proc'].apply(lambda x: get_top_n_ngrams(x, n=10, ngram=3, exclude_keywords=['chat', 'gpt', 'chatgpt','httpstcorlyimpqw', 'ai', 'eth', 'può','tipo', 'ogni', 'prima', 'dopo']))

# creo un df per metterci i trigrammi e il mese
df_top_trigrams_it = pd.DataFrame(top_trigrams_by_month_it.reset_index())

# espando la lista dei trigrammi su rows distinte
df_top_trigrams_it = df_top_trigrams_it.explode('tweetText_proc')

# Split the 'tweet_text' in trigram e frequency
df_top_trigrams_it[['Trigram', 'Frequency']] = pd.DataFrame(df_top_trigrams_it['tweetText_proc'].tolist(), index=df_top_trigrams_it.index)


df_top_trigrams_it = df_top_trigrams_it.drop(columns='tweetText_proc')


df_top_trigrams_it['Sentimento'] = df_top_trigrams_it['Trigram'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [53]:
print(df_top_trigrams_it)

    Month                              Trigram  Frequency  Sentimento
0       1                 leggi larticolo link         36         0.0
0       1    comprendi tecnologia applicazioni         34         0.0
0       1          italia comprendi tecnologia         30         0.0
0       1  intelligenza artificiale generativa         25         0.0
0       1            minaccia monopolio motore         19         0.0
..    ...                                  ...        ...         ...
9      12         ora intelligenza artificiale         11         0.0
9      12             google deve preoccuparsi         10         0.0
9      12    intelligenza artificiale stupisce         10         0.0
9      12            problema sicurezza hacker         10         0.0
9      12          sicurezza hacker potrebbero         10         0.0

[100 rows x 4 columns]


In [54]:
months_mapping = {1: 'Gennaio', 2: 'Febbraio', 3: 'Marzo', 4: 'Aprile', 5: 'Maggio', 6: 'Giugno', 9: 'Settembre', 10:'Ottobre', 11:'Novembre', 12:'Dicembre'}

df_top_trigrams['MonthName'] = df_top_trigrams['Month'].map(months_mapping)
df_top_trigrams = df_top_trigrams.sort_values('MonthName')
df_top_trigrams

,Month,Trigram,Frequency,Sentimento,MonthName
3,4,banned italy privacy,146,-0.4588,Aprile
3,4,spot recommendation ticker,86,0.0000,Aprile
3,4,recommendation ticker time,86,0.0000,Aprile
3,4,ticker time interval,86,0.0000,Aprile
3,4,special edition guide,84,0.4019,Aprile
...,...,...,...,...,...
6,9,100 3000 week,2,0.0000,Settembre
6,9,home owner charged,2,-0.2023,Settembre
6,9,flying bussing illegals,2,0.0000,Settembre
6,9,trillions dollars members,1,0.0000,Settembre


In [55]:
months_mapping = {1: 'Gennaio', 2: 'Febbraio', 3: 'Marzo', 4: 'Aprile', 5: 'Maggio', 6: 'Giugno', 9: 'Settembre', 10:'Ottobre', 11:'Novembre', 12:'Dicembre'}

df_top_trigrams_it['MonthName'] = df_top_trigrams_it['Month'].map(months_mapping)
df_top_trigrams_it = df_top_trigrams_it.sort_values('MonthName')
df_top_trigrams_it

,Month,Trigram,Frequency,Sentimento,MonthName
3,4,info qui tagga,14,0.0,Aprile
3,4,tagga persona interessata,14,0.0,Aprile
3,4,persona interessata opinione,12,0.0,Aprile
3,4,interessata opinione commenti,12,0.0,Aprile
3,4,scrivere prompt efficace,6,0.0,Aprile
...,...,...,...,...,...
6,9,fuori gioco google,3,0.0,Settembre
6,9,scopriamo cos funziona,2,0.0,Settembre
6,9,oltre milione utenti,3,0.0,Settembre
6,9,concorrenza google risposte,3,0.0,Settembre


In [101]:
#month_dropdown = alt.binding_select(options=df_top_trigrams['MonthName'].unique().tolist()+[None])
#month_sel = alt.selection_single(fields=['MonthName'], bind=month_dropdown, name="Seleziona", init={'MonthName': df_top_trigrams['MonthName'].iloc[0]})
optmont = ['Settembre','Ottobre','Dicembre','Gennaio', 'Febbraio', 'Marzo', 'Aprile', 'Maggio', 'Giugno']
labels = [option + ' ' for option in optmont]
month_dropdown = alt.binding_select(
    # Add the empty selection which shows all when clicked
    options=optmont + [None],
    labels=labels + ['Tutto'],
    name='Selettore del periodo: '
)

month_sel = alt.selection_point(
    fields=['MonthName'],
    bind=month_dropdown,
)
#sentiment_scale = alt.Scale(domain=[-1, 1], range=['#FC2E20', '#FD7F20', '#FDB750'])
#single = alt.selection_single(on='mouseover', nearest=True)
tooltip = alt.selection_single(fields=['Trigram', 'Frequency'])


test = alt.Chart(df_top_trigrams, title=alt.Title('Top trigrammi sul periodo 2022/2023',
    subtitle= 'Dati relativi ai tweets in lingua inglese')).mark_bar().encode(
    x=alt.X('Trigram:N', sort='-y', title='Trigramma'),
    y=alt.Y('Frequency:Q', title='Frequenza registrata'),
    color=alt.Color('Sentimento:Q'), #, scale=sentiment_scale),
    tooltip=[alt.Tooltip('Trigram:N', title="Trigramma"),
             alt.Tooltip('Frequency:Q', title="Frequenza")]
).properties(
    width=800,
    height=600,
).configure_axisX(
    labelAngle=45,  # Specify the degree of inclination here
    labelAlign='left',  # Optional: Align the labels to the right for better readability
    labelPadding=15  # Optional: Add padding to the labels for better spacing
).add_selection(month_sel, tooltip
).transform_filter(month_sel
).transform_window(
    rank='rank(row_number)',
    sort=[alt.SortField('Frequency', order='descending')]
).transform_filter(
    alt.datum.rank <= 30
).resolve_scale(y='independent'
).configure_legend(orient='top-right').interactive()

test_it = alt.Chart(df_top_trigrams_it, title=alt.Title('Top trigrammi sul periodo 2022/2023',
    subtitle= 'Dati relativi ai tweets in lingua italiana')).mark_bar().encode(
    x=alt.X('Trigram:N', sort='-y', title='Trigramma'),
    y=alt.Y('Frequency:Q', title='Frequenza registrata'),
    color=alt.Color('Sentimento:Q'), #, scale=sentiment_scale),
    tooltip=[alt.Tooltip('Trigram:N', title="Trigramma"),
             alt.Tooltip('Frequency:Q', title="Frequenza")]
).properties(
    width=800,
    height=600,
).configure_axisX(
    labelAngle=45,  # Specify the degree of inclination here
    labelAlign='left',  # Optional: Align the labels to the right for better readability
    labelPadding=15  # Optional: Add padding to the labels for better spacing
).add_selection(month_sel, tooltip
).transform_filter(month_sel
).transform_window(
    rank='rank(row_number)',
    sort=[alt.SortField('Frequency', order='descending')]
).transform_filter(
    alt.datum.rank <= 30
).resolve_scale(y='independent'
).configure_legend(orient='top-right').interactive()

test_it

alt.Chart(...)

In [102]:
test

alt.Chart(...)

In [104]:
test.save('trigrammimensilitwitter.html')
test_it.save('trigrammimensilitwitterIT.html')

<h2>TOPIC MODELLING, GRAFICI</h2>

In [58]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

# Read the DataFrame
dftm = pd.read_csv('dfITEN.csv')  # Replace 'your_data.csv' with your actual data file
dfredd = pd.read_csv('reddit_altair.csv')

In [59]:
dftm.head()

,Unnamed: 0,user,date,likeCount,tweetText,hashtags,lang,replies,retweetCount,replyCount,unique_id
0,0,jdemay,2023-02-05 23:59:16+00:00,0,Planning to use #ChatGPT to help you with high...,['ChatGPT'],en,0.0,0.0,NaN,0
1,1,CarlosSilvaB81,2023-02-05 23:58:51+00:00,3,"@Carmtans @stkirsch Chat gpt, is nothing but a...",NaN,en,0.0,0.0,NaN,1
2,2,saadnajeebsaad,2023-02-05 23:58:15+00:00,0,#WeatherUpdate #Karachi #Khi #TempUpdate #Sunr...,"['WeatherUpdate', 'Karachi', 'Khi', 'TempUpdat...",en,0.0,0.0,NaN,2
3,3,RyanMac69987725,2023-02-05 23:58:07+00:00,0,@MKBHD But who is in the lead? Is it Microsoft...,NaN,en,0.0,0.0,NaN,3
4,4,sjfcommo,2023-02-05 23:57:46+00:00,1,#ChatGPT seems like a great way to start creat...,['ChatGPT'],en,0.0,0.0,NaN,4


In [60]:
dfredd.head()

,Unnamed: 0,Post ID,Post author,Post title,Post URL,Post flair type,Post score,Post date,Comment ID,Comment author,Comment,Comment score,Comment date,Parent Comment ID,Parent comment author,Sentiment
0,1,13ucoev,TheHybred,If ChatGPT Cant Access The Internet Then How I...,https://www.reddit.com/r/ChatGPT/comments/13uc...,Jailbreak,4318,2023-05-29 00:10:45,jm090r5,sdmat,reason technical surprisingly nuanced Training...,2482,2023-05-29 01:42:37,13ucoev,TheHybred,pos
1,2,13ucoev,TheHybred,If ChatGPT Cant Access The Internet Then How I...,https://www.reddit.com/r/ChatGPT/comments/13uc...,Jailbreak,4318,2023-05-29 00:10:45,jm0k235,bojodrop,Slide jailbreak prompt,418,2023-05-29 03:13:53,13ucoev,TheHybred,neu
2,3,13ucoev,TheHybred,If ChatGPT Cant Access The Internet Then How I...,https://www.reddit.com/r/ChatGPT/comments/13uc...,Jailbreak,4318,2023-05-29 00:10:45,jm0h4ut,opi098514,Easy next line Shes old,631,2023-05-29 02:49:38,13ucoev,TheHybred,pos
3,4,13ucoev,TheHybred,If ChatGPT Cant Access The Internet Then How I...,https://www.reddit.com/r/ChatGPT/comments/13uc...,Jailbreak,4318,2023-05-29 00:10:45,jm0dl2p,Cryptizard,could infer trying ask question would give dif...,193,2023-05-29 02:20:05,13ucoev,TheHybred,neg
4,5,13ucoev,TheHybred,If ChatGPT Cant Access The Internet Then How I...,https://www.reddit.com/r/ChatGPT/comments/13uc...,Jailbreak,4318,2023-05-29 00:10:45,jm0pebs,manikfox,link conversation directly feature see prompts...,77,2023-05-29 03:59:01,13ucoev,TheHybred,neg


In [61]:
#polarità calcolata con vader
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
#Controllo il linguaggio e rimuovo le stopwords per caso
def get_stop_words(lang):
    if lang == 'en':
        return nltk.corpus.stopwords.words('english')
    elif lang == 'it':
        return nltk.corpus.stopwords.words('italian')
    else:
        return []

#Rimozione stopwords
def remove_stop_words(text, stop_words):
    text = text.translate(str.maketrans('', '', string.punctuation))
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\claud\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [62]:
def applier(df):
    # Apply stop words removal
    df['tweetText'] = df['tweetText'].fillna('')  #per riempire eventuali vuoti (non dovrebbero essercene)
    df['stop_words'] = df['tweetText'].apply(get_stop_words)
    df['stop_content'] = df['tweetText'].apply(lambda x: remove_stop_words(x, stop_words=df['stop_words']))
    return df

def polarity(df):
    df['vader_polarity'] = df['tweetText'].map(lambda text: sid.polarity_scores(text)['compound']) 
    #polarità con textblob
    df['blob_polarity'] = df['tweetText'].map(lambda text: TextBlob(text).sentiment.polarity)
    return df

In [63]:
def applier_reddit(df):
    # Apply stop words removal
    df['Comment'] = df['Comment'].fillna('')  #per riempire eventuali vuoti (non dovrebbero essercene)
    df['stop_words'] = df['Comment'].apply(get_stop_words)
    df['stop_content'] = df['Comment'].apply(lambda x: remove_stop_words(x, stop_words=df['stop_words']))
    return df

def polarity_reddit(df):
    df['vader_polarity'] = df['Comment'].map(lambda text: sid.polarity_scores(text)['compound']) 
    #polarità con textblob
    df['blob_polarity'] = df['Comment'].map(lambda text: TextBlob(text).sentiment.polarity)
    return df

In [64]:
dftm = polarity(applier(df))

In [65]:
dfredd_r = polarity_reddit(applier_reddit(dfredd))

In [66]:
# We define a list of topics
topics = ['browser', 'google', 'church', 'microsoft' ,'learning', 'school', 'cryptocurrencies', 'ban', 'italy ban','privacy'] #'ai', 'chatgpt', 'openai'

In [67]:
# creo una colonna chiamata topic
dfredd_r['Topic'] = ""
for topic in topics:
    dfredd_r.loc[dfredd_r['stop_content'].str.contains(topic), 'Topic'] = topic

In [68]:
# creo una colonna chiamata topic
dftm['Topic'] = ""
for topic in topics:
    dftm.loc[dftm['stop_content'].str.contains(topic), 'Topic'] = topic

In [69]:
# We create a new DataFrame with columns topic / sentiment / source
data = []
for topic in topics:
    topic_rows = dfredd_r[dfredd_r['Topic'] == topic]
    # Average sentiment per topic
    vader_sentiments = topic_rows['vader_polarity'].sum() / topic_rows.shape[0]
    textblob_sentiments = topic_rows['blob_polarity'].sum() / topic_rows.shape[0]
    # Append data
    data.append({'Topic': topic, 'Sentimento': vader_sentiments, 'Source': 'Vader'})
    data.append({'Topic': topic, 'Sentimento': textblob_sentiments, 'Source': 'TextBlob'})

df_new2 = pd.DataFrame(data)

C:\Users\claud\AppData\Local\Temp\ipykernel_27376\3410561922.py:6: RuntimeWarning: invalid value encountered in double_scalars
  vader_sentiments = topic_rows['vader_polarity'].sum() / topic_rows.shape[0]
C:\Users\claud\AppData\Local\Temp\ipykernel_27376\3410561922.py:7: RuntimeWarning: invalid value encountered in double_scalars
  textblob_sentiments = topic_rows['blob_polarity'].sum() / topic_rows.shape[0]


In [70]:
# We create a new DataFrame with columns topic / sentiment / source
data = []
for topic in topics:
    topic_rows = dftm[dftm['Topic'] == topic]
    # Average sentiment per topic
    vader_sentiments = topic_rows['vader_polarity'].sum() / topic_rows.shape[0]
    textblob_sentiments = topic_rows['blob_polarity'].sum() / topic_rows.shape[0]
    # Append data
    data.append({'Topic': topic, 'Sentimento': vader_sentiments, 'Source': 'Vader'})
    data.append({'Topic': topic, 'Sentimento': textblob_sentiments, 'Source': 'TextBlob'})

df_new = pd.DataFrame(data)

In [71]:
df_new

,Topic,Sentimento,Source
0,browser,0.125138,Vader
1,browser,0.113067,TextBlob
2,google,0.155390,Vader
3,google,0.117005,TextBlob
4,church,0.149784,Vader
5,church,0.074032,TextBlob
6,microsoft,0.181705,Vader
7,microsoft,0.106431,TextBlob
8,learning,0.291808,Vader
9,learning,0.154824,TextBlob


In [72]:
df_new2

,Topic,Sentimento,Source
0,browser,0.210277,Vader
1,browser,0.094465,TextBlob
2,google,0.240025,Vader
3,google,0.087477,TextBlob
4,church,0.148109,Vader
5,church,0.069898,TextBlob
6,microsoft,0.037917,Vader
7,microsoft,0.083878,TextBlob
8,learning,0.374545,Vader
9,learning,0.092297,TextBlob


In [73]:
valuetwitter = 'Twitter'
provadf = df_new
provadf = provadf.assign(social=valuetwitter)
#print(provadf)

In [74]:
valuereddit = 'Reddit'
provadf2 = df_new2
provadf2 = provadf2.assign(social=valuereddit)
#print(provadf2)

In [75]:
concatenated_df = pd.concat([provadf, provadf2], axis=0)

# Reset the index of the concatenated DataFrame
concatenated_df = concatenated_df.reset_index(drop=True)

In [76]:
concatenated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Topic       40 non-null     object 
 1   Sentimento  38 non-null     float64
 2   Source      40 non-null     object 
 3   social      40 non-null     object 
dtypes: float64(1), object(3)
memory usage: 1.4+ KB


In [79]:
options = ['Vader', 'TextBlob']
labels = [option for option in options]
dropsource = alt.binding_radio(
    # Add the empty selection which shows all when clicked
    options=options,
    labels=labels,
    name='Selettore libreria: '
)
#default_sel = options[0]
selsource = alt.selection_point(
    fields=['Source'],
    bind=dropsource
)
comparativeAnalysis= alt.Chart(concatenated_df).mark_bar(opacity=0.7).encode(
    alt.X("social:N").title('').axis(labels=False),
    alt.Y("Sentimento:Q").title("Sentimento"),
    alt.Color("social:N")
        .scale(domain=("Twitter", "Reddit"), range=['#1DA1F2', '#F44E3F'])
        .title("Social di riferimento"),
    alt.Column("Topic:O", title="Argomento")
        .sort(alt.SortField(field='Topic', order='descending'))
        .header(orient='bottom'),
    tooltip=['Topic', 'Sentimento', 'Source']
).properties(
    title="Analisi comparativa sui valori registrati nei dataset di Twitter e Reddit"
).configure_title(
    fontSize=16,
    fontWeight='bold'
).configure_facet(
    spacing=30
).configure_legend(orient='right'
).configure_header(
    labelAngle=45,
    labelAlign='left',
    labelPadding=10
).add_params(
    selsource
).transform_filter(
    selsource
).configure_facet(
    spacing=10
).interactive()

comparativeAnalysis.save('comparativeAnalysis_twitter_reddit.html')
comparativeAnalysis

alt.Chart(...)